### Deliverable 3: Optimize the model to get over 75% accuracy

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 500?
replace_application = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than 1800?
replace_class = list(classification_counts[classification_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [6]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes=='object'].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### **Original Model**



In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 352       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=30)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/30
804/804 [==============================] - 2s 1ms/step - loss: 0.6361 - accuracy: 0.6346
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5717 - accuracy: 0.7054
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5642 - accuracy: 0.7215
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5584 - accuracy: 0.7271
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5600 - accuracy: 0.7270
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5584 - accuracy: 0.7335
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7313
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5557 - accuracy: 0.7308
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5595 - accuracy: 0.7272
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5589 - accuracy: 0.7274

### **Optimzation Attempt 1:** Add a third and fourth hidden layer

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 6
hidden_nodes_layer3 = 4
hidden_nodes_layer4 = 2

nn_opt1 = tf.keras.models.Sequential()

# First hidden layer
nn_opt1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_opt1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn_opt1.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn_opt1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 352       
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 3         
Total params: 447
Trainable params: 447
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn_opt1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt1 = nn_opt1.fit(X_train_scaled,y_train,epochs=30)

# Evaluate the model using the test data
model_loss_opt1, model_accuracy_opt1 = nn_opt1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_opt1}, Accuracy: {model_accuracy_opt1}")

Epoch 1/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6537 - accuracy: 0.6583
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5684 - accuracy: 0.7275
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5615 - accuracy: 0.7302
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5581 - accuracy: 0.7300
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5570 - accuracy: 0.7302
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7339
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5577 - accuracy: 0.7267
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7310
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accuracy: 0.7334
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7295

### **Optimzation Attempt 2:** Add neurons to hidden layers 

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 25

nn_opt2 = tf.keras.models.Sequential()

# First hidden layer
nn_opt2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_opt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 50)                2200      
_________________________________________________________________
dense_9 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 26        
Total params: 3,501
Trainable params: 3,501
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn_opt2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt2 = nn_opt2.fit(X_train_scaled,y_train,epochs=30)

# Evaluate the model using the test data
model_loss_opt2, model_accuracy_opt2 = nn_opt2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_opt2}, Accuracy: {model_accuracy_opt2}")

Epoch 1/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5975 - accuracy: 0.6996
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5521 - accuracy: 0.7334
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7334
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7313
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7293
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5495 - accuracy: 0.7337
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7348
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5430 - accuracy: 0.7372
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7331
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7343

### **Optimzation Attempt 3:** Change Activation Function 

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_opt3 = tf.keras.models.Sequential()

# First hidden layer
nn_opt3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn_opt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn_opt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 8)                 352       
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 6         
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn_opt3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt3 = nn_opt3.fit(X_train_scaled,y_train,epochs=30)

# Evaluate the model using the test data
model_loss_opt3, model_accuracy_opt3 = nn_opt3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_opt3}, Accuracy: {model_accuracy_opt3}")

Epoch 1/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6576 - accuracy: 0.6171
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5667 - accuracy: 0.7286
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5620 - accuracy: 0.7280
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5562 - accuracy: 0.7336
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7313
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5568 - accuracy: 0.7306
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7285
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5514 - accuracy: 0.7322
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.7292
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7316

### **Optimzation Attempt 4:** Adjust Input Data (Drop INCOME_AMT)

In [18]:
application_opt4_df = application_df.drop(['INCOME_AMT_0', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999', 
                                      'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M', 
                                      'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M'],1)

In [20]:
# Split our preprocessed data into our features and target arrays
y_opt4 = application_opt4_df["IS_SUCCESSFUL"].values
X_opt4 = application_opt4_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train_opt4, X_test_opt4, y_train_opt4, y_test_opt4 = train_test_split(X_opt4, y_opt4, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler_opt4 = scaler.fit(X_train_opt4)

# Scale the data
X_train_scaled_opt4 = X_scaler_opt4.transform(X_train_opt4)
X_test_scaled_opt4 = X_scaler_opt4.transform(X_test_opt4)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_opt4[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_opt4 = tf.keras.models.Sequential()

# First hidden layer
nn_opt4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_opt4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_opt4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 8)                 280       
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 6         
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn_opt4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_opt4 = nn_opt4.fit(X_train_scaled_opt4,y_train_opt4,epochs=30)

# Evaluate the model using the test data
model_loss_opt4, model_accuracy_opt4 = nn_opt4.evaluate(X_test_scaled_opt4,y_test_opt4,verbose=2)
print(f"Loss: {model_loss_opt4}, Accuracy: {model_accuracy_opt4}")

Epoch 1/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6970 - accuracy: 0.6117
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5908 - accuracy: 0.7077
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5722 - accuracy: 0.7252
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5677 - accuracy: 0.7254
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5618 - accuracy: 0.7302
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5657 - accuracy: 0.7214
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5609 - accuracy: 0.7250
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5623 - accuracy: 0.7266
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5618 - accuracy: 0.7273
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5599 - accuracy: 0.7290

In [23]:
print(f"Original Model - Loss: {model_loss:,.4f}, Accuracy: {model_accuracy:,.4f}")
print(f"Optimization 1 - Loss: {model_loss_opt1:,.4f}, Accuracy: {model_accuracy_opt1:,.4f}")
print(f"Optimization 2 - Loss: {model_loss_opt2:,.4f}, Accuracy: {model_accuracy_opt2:,.4f}")
print(f"Optimization 3 - Loss: {model_loss_opt3:,.4f}, Accuracy: {model_accuracy_opt3:,.4f}")
print(f"Optimization 4 - Loss: {model_loss_opt4:,.4f}, Accuracy: {model_accuracy_opt4:,.4f}")

Original Model - Loss: 0.5571, Accuracy: 0.7237
Optimization 1 - Loss: 0.5520, Accuracy: 0.7280
Optimization 2 - Loss: 0.5528, Accuracy: 0.7270
Optimization 3 - Loss: 0.5525, Accuracy: 0.7268
Optimization 4 - Loss: 0.5606, Accuracy: 0.7230
